In [1]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping


2023-03-22 13:01:08.871297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_train = np.load('Data/x_train_sample.npy')/255
y_train = np.load('Data/y_train_sample.npy')
x_test = np.load('Data/x_test_sample.npy')/255
y_test = np.load('Data/y_test_sample.npy')

In [3]:
print(x_train.shape)
print(y_train.shape)

(1200, 32, 32, 3)
(1200, 15)


https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798
https://gist.github.com/mstfldmr/44dfce35f5330b22ee1e3b28ca91a3e7
https://medium.com/analytics-vidhya/denoising-autoencoder-on-colored-images-using-tensorflow-17bf63e19dad

In [4]:
input_size = 1200
hidden_size = 256
code_size = 64

input_img = Input(shape=(32,32,3,))
hidden_1 = Dense(hidden_size, activation='relu')(input_img)
code = Dense(code_size, activation='relu')(hidden_1)
hidden_2 = Dense(hidden_size, activation='relu')(code)
output_img = Dense(input_size, activation='sigmoid')(hidden_2)



2023-03-22 13:01:11.214097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def create_model():
  x = Input(shape=(32, 32, 3))
# Encoder
  e_conv1 = Conv2D(hidden_size, (3, 3), activation='relu', padding='same')(x)
  pool1 = MaxPooling2D((2, 2), padding='same')(e_conv1)
  batchnorm_1 = BatchNormalization()(pool1)
  e_conv2 = Conv2D(code_size, (3, 3), activation='relu', padding='same')(batchnorm_1)
  pool2 = MaxPooling2D((2, 2), padding='same')(e_conv2)
  batchnorm_2 = BatchNormalization()(pool2)
  e_conv3 = Conv2D(code_size/2, (3, 3), activation='relu', padding='same')(batchnorm_2)
  h = MaxPooling2D((2, 2), padding='same')(e_conv3)
# Decoder
  d_conv1 = Conv2D(hidden_size, (3, 3), activation='relu', padding='same')(h)
  up1 = UpSampling2D((2, 2))(d_conv1)
  d_conv2 = Conv2D(code_size, (3, 3), activation='relu', padding='same')(up1)
  up2 = UpSampling2D((2, 2))(d_conv2)
  d_conv3 = Conv2D(code_size/2, (3, 3), activation='relu')(up2)
  up3 = UpSampling2D((2, 2))(d_conv3)
  r = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up3)
  model = Model(x, r)
  model.compile(optimizer='adam', loss='mse')
  return model
auto_encoder = create_model()
early_stop = EarlyStopping(monitor='loss', patience=3)
history = auto_encoder.fit(x_train, x_train, epochs=50, batch_size=32, callbacks=[early_stop])

Epoch 1/50


ValueError: in user code:

    File "/Users/xin/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/xin/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/xin/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/xin/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/xin/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/Users/xin/opt/anaconda3/e

In [ ]:
model.add(Conv2D(3,(3, 3), padding='same'))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 1200)      33600     
                                                                 
 activation (Activation)     (None, 32, 32, 1200)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 1200)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        691264    
                                                                 
 activation_1 (Activation)   (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

In [ ]:
model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
model.fit(x_train,x_train)

38/38 [==============================] - 49s 1s/step - loss: 0.6931


In [ ]:
x_train.shape

(1200, 32, 32, 3)

In [6]:
model = Sequential()

In [7]:
#1st convolution layer
model.add(Conv2D(512, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

#2nd convolution layer
model.add(Conv2D(128,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
#-------------------------
#3rd convolution layer
model.add(Conv2D(128,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(UpSampling2D((2, 2)))

#4rd convolution layer
model.add(Conv2D(512,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(UpSampling2D((2, 2)))

In [8]:
model.add(Conv2D(3,(3, 3), padding='same'))
model.add(Activation('sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 32, 32, 512)       14336     
                                                                 
 activation (Activation)     (None, 32, 32, 512)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 512)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 128)       589952    
                                                                 
 activation_1 (Activation)   (None, 16, 16, 128)       0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                    

In [10]:
model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [11]:
model.fit(x_train,x_train,batch_size=50,epochs=5)

Epoch 1/5
24/24 [==============================] - 31s 1s/step - loss: 0.6931
Epoch 2/5
24/24 [==============================] - 33s 1s/step - loss: 0.6929
Epoch 3/5
24/24 [==============================] - 34s 1s/step - loss: 0.6926
Epoch 4/5
24/24 [==============================] - 32s 1s/step - loss: 0.6923
Epoch 5/5
24/24 [==============================] - 33s 1s/step - loss: 0.6920
